In [11]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable

In [3]:
trYs = torch.load("../TrainData_traits.pt")
trXgenos = torch.load("../TrainData_genos.pt")


In [5]:
class Mydataset(Dataset):
    
    def __init__(self,trYs,trXgenos,transform=None):
        self.traits = trYs
        self.trXgenos = trXgenos
        
        assert len(self.traits) == len(self.trXgenos), "samples not match"
        
        self.transform = transform
        
        self.sample_set = []
        for idx in range(len(self.trXgenos)):
            self.sample_set.append((self.trXgenos[idx],self.traits[idx]))
            
    def __getitem__(self, index):
        genes,traits = self.sample_set[index]
        if self.transform is not None:
            genes = self.transform(genes)
            
        return genes,traits
    
    def __len__(self):
        return len(self.trXgenos)
    
trainData = Mydataset(trYs,trXgenos)
testData = Mydataset(trYs,trXgenos)

In [20]:
batch_size = 64
trainLoader = DataLoader(dataset=trainData,batch_size=64,shuffle=True)
testLoader = DataLoader(dataset=trainData,batch_size=64,shuffle=False)

trainLoader.dataset

In [23]:
class AlexNet(nn.Module):
    def __init__(self,init_weights=False):
        super(AlexNet, self).__init__()


        '''
        
        data size:3channel x32x32
        1st convolutional layer, input channel=3, output channel=96, kernel size=3x3, step=1*1'''

        '''feature map size=(32-3)/1+1=30,feature map dimension 96*30*30'''

        self.features = nn.Sequential(
            nn.Conv1d(1,96, kernel_size=3, stride=1), # output channel: pic size;
            nn.ReLU(inplace=True),
            #nn.BatchNorm1d(96), # normalization

            nn.MaxPool1d(kernel_size=3, stride=2),  # kernel 3x3; step 2x2 -> (30-3)/2 + 1 = 14x14

            nn.Conv1d(96, 256, kernel_size=(3,3), stride=(1,1)), # (14-3)/1 + 1 = 12 x 12
            nn.ReLU(inplace=True),
            #nn.BatchNorm1d(256),

            nn.MaxPool1d(kernel_size=3, stride=2),  # (12 - 3) /2 +1 = 5x5

            nn.Conv1d(256, 384, kernel_size=3, padding=1, stride=1),  # (5-3+2*1)/1+1=5x5
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 384, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),  # (5-3)/2+1=2
        )

        self.classifier = nn.Sequential(

            nn.Linear(1024, 2048),

            nn.Dropout(0.5),

            nn.Linear(2048, 2048),

            nn.Dropout(0.5),

            nn.Linear(2048, 1)
        )
        if init_weights:
            self._initialize_weights()


    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')  
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.features(x)

        out = torch.flatten(out,start_dim=1)

        #out = out.reshape(-1, 256 * 2 * 2)

        out = self.classifier(out)

        """
        out = F.relu(self.linear1(out))

        out = self.dropout1(out)

        out = F.relu(self.linear2(out))

        out = self.dropout2(out)

        out = self.linear3(out)
        """
        return out
    
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss, correct = 0, 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader): # enumerate

            images=Variable(images)
            labels=Variable(labels)
            #print(labels)
            # Forward pass
            #outputs = model(images)
            #loss = criterion(outputs, labels)

            X, y = images.to(device), labels.to(device)
            pred = model(X)
            predict = torch.max(pred,dim=1)[1]
            #print("##################")
            #print(predict)
            test_loss += loss_fn(pred, y).item()
            correct += (predict == y).sum().item()
            #correct += (pred == y).type(torch.max).sum().item()
            size += len(labels)
        correct /= size
        test_loss /= num_batches

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = AlexNet().to(device)

#summary.summary(model, input_size=(3, 32, 32), batch_size=128, device="cpu")

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

total_step = len(trainLoader)
epochs = 30
for epoch in range(epochs):
    model.train()
    i=0
    for i,(images, labels) in enumerate(trainLoader):
        optimizer.zero_grad()
        images=Variable(images).to(device)
        labels=Variable(labels).to(device)
        # Forward pass
        outputs = model(images)

        loss = criterion(outputs, labels)

        # Backward and optimize
        #optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, epochs, i + 1, total_step, loss.item()))
        i += 1
    #print("temp train results: ")
    #print(torch.max(outputs, dim=1)[1])
    #print(labels)
    model.eval()
    print("test")
    test(testLoader,model,criterion)


Using cuda device


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [96, 1, 3], but got 2-dimensional input of size [64, 26086] instead